In [283]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from subprocess import check_output

# Pipeline at a glance:
load data -> cleaning/preprocessing -> feature eng -> format data for modelling -> fit model -> evaluate model -> generate submission file

## Step 1: Load data

In [284]:
df_train = pd.read_csv('../input/train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [285]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [286]:
df_test = pd.read_csv('../input/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [287]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [288]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Step 2: Basic cleaning and preprocessing

In [289]:
# label encode sex
le = LabelEncoder()
le.fit(df_train.Sex)
df_train.loc[:,'Sex'] = le.transform(df_train.Sex)
df_test.loc[:,'Sex'] = le.transform(df_test.Sex)

In [290]:
values = {"Embarked":"S"}
df_train.fillna(value=values, inplace=True)
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [291]:
# Fill in the missing values
mAge = pd.concat((df_train.Age, df_test.Age), axis=0).mean()
df_train.loc[:, 'Age'] = df_train.Age.fillna(mAge)
df_test.loc[:, 'Age'] = df_test.Age.fillna(mAge)

## Step 3: Feature engineering

In [292]:
df_train['age_binned'] = pd.cut(df_train['Age'], [0, 16, 32, 48, 64, 200], labels=[0,1,2,3,4], retbins=False)
df_test['age_binned'] = pd.cut(df_test['Age'], [0, 16, 32, 48, 64, 200], labels=[0,1,2,3,4], retbins=False)

In [293]:
# Add feature family_size
train_family_size = df_train['SibSp'] + df_train['Parch']
df_train['family_size'] = pd.Series(data=train_family_size)
test_family_size = df_test['SibSp'] + df_test['Parch']
df_test['family_size'] = pd.Series(data=test_family_size)

In [294]:
# Add feature is_alone
train_is_alone = (df_train['family_size']) < 1
df_train['is_alone'] = pd.Series(data=train_is_alone)
test_is_alone = (df_test['family_size']) < 1
df_test['is_alone'] = pd.Series(data=test_is_alone)

In [295]:
# label encode is_alone
le = LabelEncoder()
le.fit(df_train['is_alone'])
df_train['is_alone'] = le.transform(df_train['is_alone'])
df_test['is_alone'] = le.transform(df_test['is_alone'])

In [296]:
# label encode embarked
le.fit(df_train['Embarked'])
df_train['Embarked'] = le.transform(df_train['Embarked'])
df_test['Embarked'] = le.transform(df_test['Embarked'])

In [297]:
# Drop all the columns except for Survived, pclass, sex, age_binned, fam_size, is_alone, embarked.
df_train = df_train.drop(columns=['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'])
df_test = df_test.drop(columns=['Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'])

In [300]:
df_train.head()

,Survived,Pclass,Sex,Embarked,age_binned,family_size,is_alone
0,0,3,1,2,1,1,0
1,1,1,0,0,2,1,0
2,1,3,0,2,1,0,1
3,1,1,0,2,2,1,0
4,0,3,1,2,2,0,1


## Step 4: Format data for modelling

In [301]:
# Split data into X and y, and select features to use
X = df_train.loc[:,['Pclass', 'Sex', 'Embarked', 'age_binned', 'family_size', 'is_alone']]
y = df_train.loc[:,'Survived']

# OneHotEncode categorical data
enc = OneHotEncoder(categorical_features=[0, 1, 2, 3, 5])
X = enc.fit_transform(X)

In [302]:
# Split data X and y into train and val sets
X_test = df_test.loc[:,['Pclass', 'Sex', 'Embarked', 'age_binned', 'family_size', 'is_alone']]
X_test = enc.fit_transform(X_test)
X_train, X_valid, y_train, y_valid = train_test_split(
      X, y, test_size=0.2, random_state=42)

## Step 5: Train (or fit) the model

In [303]:
# Fit logistic regression
#LR = LogisticRegression()
#LR.fit(X_train, y_train)

In [304]:
# Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Step 6: Evaluate the model

In [305]:
# define a scoring function
def acc(y: np.array, y_pred: np.array) -> float:
    return np.sum(y_pred==y)/len(y)

In [306]:
# Use model to predict validation set
#y_pred_valid = LR.predict(X_valid)

In [307]:
# Fit Random Forest Classifier to X_valid
y_pred_valid = classifier.predict(X_valid)

In [308]:
print('Validation set accuracy', acc(y_valid, y_pred_valid), '%')

Validation set accuracy 0.8156424581005587 %


## Step 7: Generate Submission

In [309]:
df_test.head()

,PassengerId,Pclass,Sex,Embarked,age_binned,family_size,is_alone
0,892,3,1,1,2,0,1
1,893,3,0,2,2,1,0
2,894,2,1,1,3,0,1
3,895,3,1,2,1,0,1
4,896,3,0,2,1,2,0


In [310]:
# Predict for test set

y_pred_test = LR.predict(X_test)

# Create a Kaggle submission
sub = pd.DataFrame({'PassengerId': df_test['PassengerId'],
                    'Survived': y_pred_test})
sub.head()

sub.to_csv('basic_submission.csv', index=False)